In [33]:
import pandas as pd
import tensorflow.keras.layers as tfl
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [34]:
train = pd.read_csv("data/train_prepared.csv")
test = pd.read_csv("data/test_prepared.csv")

In [35]:
train.columns

Index(['id', 'Rating', 'maincateg', 'platform', 'price1', 'actprice1',
       'norating1', 'noreviews1', 'star_5f', 'star_4f', 'star_3f', 'star_2f',
       'star_1f', 'fulfilled1', 'combo', 'category'],
      dtype='object')

In [36]:
train.drop("id", axis=1, inplace=True)
test_id = test.id
test.drop("id", axis=1, inplace=True)

X = train.drop(["price1"], axis=1)
y = train["price1"]
assert X.shape[1] == test.shape[1], "X and test have different number of columns"

In [37]:
columns_to_one_hot = ["maincateg", "category", "platform", "combo"]
columns_to_standardize = ["actprice1", 'norating1', 'noreviews1', 'star_5f', 
    'star_4f', 'star_3f', 'star_2f', 'star_1f',]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse=False)
standardizer = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("one_hot_encoder", one_hot_encoder, columns_to_one_hot),
        # ("standardizer", standardizer, columns_to_standardize)
        ],
    remainder="passthrough"
)

In [41]:
preprocessor.fit(X_train)
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)
test = preprocessor.transform(test)

In [42]:
assert X_train.shape[1] == test.shape[1], "X and test have different number of columns"

In [43]:
input_shape = X_test.shape[1]

In [44]:
input_ = tfl.Input(shape=(input_shape,))
x = tfl.Dense(units=256, activation="relu")(input_)
x = tfl.Dense(units=128, activation="relu")(x)
x = tfl.Dense(units=64, activation="relu")(x)
x = tfl.Dense(units=32, activation="relu")(x)
x = tfl.Dense(units=16, activation="relu")(x)
x = tfl.Dense(units=8, activation="relu")(x)
output = tfl.Dense(units=1, activation="linear")(x)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 27)]              0         
                                                                 
 dense_13 (Dense)            (None, 256)               7168      
                                                                 
 dense_14 (Dense)            (None, 128)               32896     
                                                                 
 dense_15 (Dense)            (None, 64)                8256      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 16)                528       
                                                                 
 dense_18 (Dense)            (None, 8)                 136 

In [45]:
model.compile(loss="mse", optimizer="adam", metrics=["mse", "mae"])

In [46]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
394/394 [==============================] - 12s 19ms/step - loss: 138722.3594 - mse: 138722.3594 - mae: 232.1292 - val_loss: 115599.1484 - val_mse: 115599.1484 - val_mae: 209.2851
Epoch 2/20
394/394 [==============================] - 6s 16ms/step - loss: 117456.4766 - mse: 117456.4766 - mae: 223.0858 - val_loss: 142779.5625 - val_mse: 142779.5625 - val_mae: 238.4545
Epoch 3/20
394/394 [==============================] - 4s 9ms/step - loss: 112137.0625 - mse: 112137.0625 - mae: 217.8696 - val_loss: 110814.3906 - val_mse: 110814.3906 - val_mae: 204.1973
Epoch 4/20
394/394 [==============================] - 4s 10ms/step - loss: 106752.9062 - mse: 106752.9062 - mae: 211.1568 - val_loss: 108026.6406 - val_mse: 108026.6406 - val_mae: 214.3644
Epoch 5/20
394/394 [==============================] - 4s 10ms/step - loss: 97771.5781 - mse: 97771.5781 - mae: 201.7287 - val_loss: 97976.7031 - val_mse: 97976.7031 - val_mae: 188.3756
Epoch 6/20
394/394 [==============================] - 4s 10

In [28]:
preds = model.predict(test)
preds = tf.squeeze(preds).numpy()
preds

164/164 [==============================] - 7s 4ms/step


array([416.96716, 315.97168, 493.39142, ..., 432.57184, 260.3942 ,
       657.8312 ], dtype=float32)

In [29]:
sample = pd.read_csv('data/Sample__submission.csv')

In [31]:
sample.price1 = preds
sample.to_csv('data/submission_final.csv', index=False)

In [32]:
!kaggle competitions submit -c sa2022 -f data/submission_final.csv -m ""

Successfully submitted to Final Capstone Project



  0%|          | 0.00/82.3k [00:00<?, ?B/s]
 10%|▉         | 8.00k/82.3k [00:00<00:01, 60.3kB/s]
100%|██████████| 82.3k/82.3k [00:05<00:00, 14.4kB/s]
